# 作业二：频繁模式与关联规则挖掘

**姓名**：王英杰  
**学号**：5720201111   

**GitHub**: <https://github.com/YingjieWang0311/data-mining>

# 目录

### 1. 处理数据集
  * 1.1 加载数据集  
  * 1.2 处理数据

### 2. 找出频繁项集
  * 2.1 算法实现
  * 2.2 找出频繁项集

### 3. 导出关联规则
  * 3.1 定义关联规则类
  * 3.2 算法实现
  * 3.3 找出关联规则

### 4. 评价关联规则
  * Lift指标
  * Conviction指标

In [1]:
import pandas as pd
import itertools
import collections
import numbers
import typing

# 1. 处理数据集

## 1.1 加载数据集


使用Oakland Crime Statistics 2011 to 2016数据集。

数据集主页：https://www.kaggle.com/cityofoakland/oakland-crime-statistics-2011-to-2016

In [2]:
df = pd.read_csv('records-for-2016.csv')
print('%d columns' % len(df.columns))
print('%d rows' % len(df.index))

10 columns
110828 rows


数据集中共有10列，110828行。数据集中的前5行展示如下。

In [3]:
df.head()

,Agency,Create Time,Location,Area Id,Beat,Priority,Incident Type Id,Incident Type Description,Event Number,Closed Time
0,OP,2016-01-01T00:00:57.000,ST&MARKET ST,P1,05X,2.0,415GS,415 GUNSHOTS,LOP160101000003,2016-01-01T00:32:30.000
1,OP,2016-01-01T00:01:25.000,AV&HAMILTON ST,P3,26Y,2.0,415GS,415 GUNSHOTS,LOP160101000005,2016-01-01T00:48:23.000
2,OP,2016-01-01T00:01:43.000,ST&CHESTNUT ST,P1,02X,2.0,415GS,415 GUNSHOTS,LOP160101000008,2016-01-01T00:21:24.000
3,OP,2016-01-01T00:01:48.000,WALLACE ST,P2,18Y,2.0,415GS,415 GUNSHOTS,LOP160101000007,2016-01-01T01:15:03.000
4,OP,2016-01-01T00:02:05.000,90TH AV,P3,34X,2.0,415GS,415 GUNSHOTS,LOP160101000009,2016-01-01T00:54:52.000


## 1.2 处理数据

选取数据集中的，`Area Id`，`Beat`，`Incident Type Id`这3列造transaction。

为了方便查看，在每一项数据的前面加上列的名称。处理后的transaction如下所示。共有110828条transaction。

In [4]:
columns = [
    'Area Id',
    'Beat',
    'Incident Type Id']
df = df[columns]

transactions = df.values.tolist()

print('%d transactions.' % len(transactions), end='\n\n')
for idx, t in enumerate(transactions[:5]):
    print('%d:' % idx, t, end='\n\n')

110828 transactions.

0: ['P1', '05X', '415GS']

1: ['P3', '26Y', '415GS']

2: ['P1', '02X', '415GS']

3: ['P2', '18Y', '415GS']

4: ['P3', '34X', '415GS']



# 2. 找出频繁项集

## 2.1 算法实现

实现Apriori算法中找频繁项集部分。核心函数为`find_itemsets()`。

参数`transactions`表示传入的transactions列表，`min_support`表示最小频繁项集要满足的最小支持度。

In [5]:
def join_step(itemsets: typing.List[tuple]):
    i = 0
    while i < len(itemsets):
        skip = 1
        *itemset_first, itemset_last = itemsets[i]
        tail_items = [itemset_last]
        tail_items_append = tail_items.append  # Micro-optimization
        for j in range(i + 1, len(itemsets)):
            *itemset_n_first, itemset_n_last = itemsets[j]
            if itemset_first == itemset_n_first:
                tail_items_append(itemset_n_last)
                skip += 1
            else:
                break
        itemset_first = tuple(itemset_first)
        for a, b in sorted(itertools.combinations(tail_items, 2)):
            yield itemset_first + (a,) + (b,)
        i += skip


def prune_step(
    itemsets: typing.List[tuple], 
    possible_itemsets: typing.List[tuple]
):
    itemsets = set(itemsets)
    for possible_itemset in possible_itemsets:
        for i in range(len(possible_itemset) - 2):
            removed = possible_itemset[:i] + possible_itemset[i + 1 :]
            if removed not in itemsets:
                break
        else:
            yield possible_itemset


def apriori_gen(itemsets: typing.List[tuple]):
    possible_extensions = join_step(itemsets)
    yield from prune_step(itemsets, possible_extensions)


def find_itemsets(
    transactions: typing.List[tuple],
    min_support: float,
    max_length: int = 8,
    verbosity: int = 0,
):
    transaction_sets = [set(t) for t in transactions if len(t) > 0]
    transactions = transaction_sets
    use_transaction = collections.defaultdict(lambda: True)

    if verbosity > 0:
        print("Generating itemsets.")
        print(" Counting itemsets of length 1.")

    counts = collections.defaultdict(int)
    num_transactions = 0
    for transaction in transactions:
        num_transactions += 1  # Increment counter for transactions
        for item in transaction:
            counts[item] += 1  # Increment counter for single-item itemsets

    large_itemsets = [
        (i, c)
        for (i, c) in counts.items()
        if (c / num_transactions) >= min_support
    ]

    if verbosity > 0:
        num_cand, num_itemsets = len(counts.items()), len(large_itemsets)
        print("  Found {} candidate itemsets of length 1.".format(num_cand))
        print("  Found {} large itemsets of length 1.".format(num_itemsets))

    # If large itemsets were found, convert to dictionary
    if large_itemsets:
        large_itemsets = {1: {(i,): c for (i, c) in sorted(large_itemsets)}}
    else:
        return dict(), num_transactions

    issubset = set.issubset  # Micro-optimization
    k = 2
    while large_itemsets[k - 1] and (max_length != 1):
        if verbosity > 0:
            print(" Counting itemsets of length {}.".format(k))
        itemsets_list = list(large_itemsets[k - 1].keys())
        C_k = list(apriori_gen(itemsets_list))
        C_k_sets = [set(itemset) for itemset in C_k]

        if verbosity > 0:
            print("  Found {} candidate itemsets of length {}.".format(len(C_k), k))
        if not C_k:
            break

        # Prepare counts of candidate itemsets (from the pruen step)
        candidate_itemset_counts = collections.defaultdict(int)
        if verbosity > 1:
            print("    Iterating over transactions.")
        for row, transaction in enumerate(transactions):
            if not use_transaction[row]:
                continue
            found_any = False
            for candidate, candidate_set in zip(C_k, C_k_sets):
                if issubset(candidate_set, transaction):
                    candidate_itemset_counts[candidate] += 1
                    found_any = True
            if not found_any:
                use_transaction[row] = False

        C_k = [
            (i, c)
            for (i, c) in candidate_itemset_counts.items()
            if (c / num_transactions) >= min_support
        ]
        if not C_k:
            break

        large_itemsets[k] = {i: c for (i, c) in sorted(C_k)}

        if verbosity > 0:
            num_found = len(large_itemsets[k])
            pp = "  Found {} large itemsets of length {}.".format(num_found, k)
            print(pp)
        k += 1
        if k > max_length:
            break

    if verbosity > 0:
        print("Itemset generation terminated.\n")
    return large_itemsets, num_transactions

## 2.2 找出频繁项集

找出支持度大于等于0.03的项集

In [6]:
itemsets, num_trans = find_itemsets(
    transactions=transactions,
    min_support=0.03,
    verbosity=1,)

Generating itemsets.
 Counting itemsets of length 1.
  Found 309 candidate itemsets of length 1.
  Found 17 large itemsets of length 1.
 Counting itemsets of length 2.
  Found 136 candidate itemsets of length 2.
  Found 10 large itemsets of length 2.
 Counting itemsets of length 3.
  Found 0 candidate itemsets of length 3.
Itemset generation terminated.



打印找到的频繁项集

In [7]:
for k, itemset in itemsets.items():
    print('%s itemsets:' % k)
    for items in itemset:
        print(items)
    print()

1 itemsets:
('04X',)
('08X',)
('10851',)
('19X',)
('242',)
('26Y',)
('30X',)
('30Y',)
('415',)
('415C',)
('5150',)
('911H',)
('933R',)
('P1',)
('P2',)
('P3',)
('SECCK',)

2 itemsets:
('04X', 'P1')
('08X', 'P1')
('19X', 'P2')
('26Y', 'P3')
('30X', 'P3')
('30Y', 'P3')
('415', 'P3')
('933R', 'P1')
('933R', 'P3')
('P1', 'SECCK')



# 3. 导出关联规则

## 3.1 定义关联规则类

首先定义关联规则类`Rule`来表示关联规则。

In [8]:
class Rule(object):
    _decimals = 3

    def __init__(
        self,
        lhs: tuple,
        rhs: tuple,
        count_full: int = 0,
        count_lhs: int = 0,
        count_rhs: int = 0,
        num_transactions: int = 0,
    ):
        self.lhs = lhs  # antecedent
        self.rhs = rhs  # consequent
        self.count_full = count_full
        self.count_lhs = count_lhs
        self.count_rhs = count_rhs
        self.num_transactions = num_transactions

    @property
    def confidence(self):
        try:
            return self.count_full / self.count_lhs
        except ZeroDivisionError:
            return None
        except AttributeError:
            return None

    @property
    def support(self):
        try:
            return self.count_full / self.num_transactions
        except ZeroDivisionError:
            return None
        except AttributeError:
            return None

    @staticmethod
    def _pf(s):
        return "{" + ", ".join(str(k) for k in s) + "}"

    def __str__(self):
        return "{} -> {}".format(self._pf(self.lhs), self._pf(self.rhs))

    def __eq__(self, other):
        return (set(self.lhs) == set(other.lhs)) and (
            set(self.rhs) == set(other.rhs)
        )

    def __hash__(self):
        return hash(frozenset(self.lhs + self.rhs))

    def __len__(self):
        return len(self.lhs + self.rhs)

## 3.2 算法实现

实现Apriori算法中找关联规则部分。核心函数为`generate_rules_apriori()`，

参数`itemsets`表示找到的频繁项集，`min_confidence`表示关联规则的最小置信度。

In [9]:
def generate_rules_apriori(
    itemsets: typing.Dict[int, typing.Dict[tuple, int]],
    min_confidence: float,
    num_transactions: int,
    verbosity: int = 0,
):
    def count(itemset):
        return itemsets[len(itemset)][itemset]

    if verbosity > 0:
        print("Generating rules from itemsets.")

    for size in itemsets.keys():
        if size < 2:
            continue
        if verbosity > 0:
            print(" Generating rules of size {}.".format(size))

        for itemset in itemsets[size].keys():
            for removed in itertools.combinations(itemset, 1):
                lhs = set(itemset).difference(set(removed))
                lhs = tuple(sorted(list(lhs)))
                conf = count(itemset) / count(lhs)
                if conf >= min_confidence:
                    yield Rule(
                        lhs,
                        removed,
                        count(itemset),
                        count(lhs),
                        count(removed),
                        num_transactions,
                    )

            H_1 = list(itertools.combinations(itemset, 1))
            yield from _ap_genrules(
                itemset, H_1, itemsets, min_confidence, num_transactions
            )

    if verbosity > 0:
        print("Rule generation terminated.\n")


def _ap_genrules(
    itemset: tuple,
    H_m: typing.List[tuple],
    itemsets: typing.Dict[int, typing.Dict[tuple, int]],
    min_conf: float,
    num_transactions: int,
):

    def count(itemset):
        return itemsets[len(itemset)][itemset]

    if len(itemset) <= (len(H_m[0]) + 1):
        return

    H_m = list(apriori_gen(H_m))
    H_m_copy = H_m.copy()

    for h_m in H_m:
        lhs = tuple(sorted(list(set(itemset).difference(set(h_m)))))
        if (count(itemset) / count(lhs)) >= min_conf:
            yield Rule(
                lhs,
                h_m,
                count(itemset),
                count(lhs),
                count(h_m),
                num_transactions,
            )
        else:
            H_m_copy.remove(h_m)
    if H_m_copy:
        yield from _ap_genrules(
            itemset, H_m_copy, itemsets, min_conf, num_transactions
        )

## 3.3 找出关联规则

找出关联规则，设置最小支持度为0.08。

In [10]:
rules = generate_rules_apriori(
    itemsets=itemsets, 
    min_confidence=0.08, 
    num_transactions=num_trans, 
    verbosity=1,
)
rules = list(rules)

Generating rules from itemsets.
 Generating rules of size 2.
Rule generation terminated.



In [11]:
rules_t = []
for rule in rules:
    valid = True
    for val in rule.lhs + rule.rhs:
        if 'nan' in val:
            valid = False
            break
    if valid:
        rules_t.append(rule)
rules = rules_t

rules_df = pd.DataFrame({
    'Rules': rules,
    'Left': list(map(lambda x: x.lhs, rules)),    
    'Right': list(map(lambda x: x.rhs, rules)),
    'Support': list(map(lambda x: x.support, rules)),
    'Confidence': list(map(lambda x: x.confidence, rules)),
})
rules_df

,Rules,Left,Right,Support,Confidence
0,{P1} -> {04X},"(P1,)","(04X,)",0.040396,0.108090
1,{04X} -> {P1},"(04X,)","(P1,)",0.040396,0.991584
2,{P1} -> {08X},"(P1,)","(08X,)",0.035027,0.093725
3,{08X} -> {P1},"(08X,)","(P1,)",0.035027,0.987535
4,{P2} -> {19X},"(P2,)","(19X,)",0.030696,0.173483
5,{19X} -> {P2},"(19X,)","(P2,)",0.030696,0.984660
6,{26Y} -> {P3},"(26Y,)","(P3,)",0.030516,0.963258
7,{30X} -> {P3},"(30X,)","(P3,)",0.030047,0.974824
8,{30Y} -> {P3},"(30Y,)","(P3,)",0.030615,0.976965
9,{415} -> {P3},"(415,)","(P3,)",0.033205,0.466827


# 4. 评价关联规则

## 4.1 Lift指标

In [12]:
def lift(rule):
    observed_support = rule.count_full / rule.num_transactions
    prod_counts = rule.count_lhs * rule.count_rhs
    expected_support = (prod_counts) / rule.num_transactions ** 2
    return observed_support / expected_support

In [13]:
rules_df['Lift'] = list(map(lambda x: lift(x), rules))
rules_df

,Rules,Left,Right,Support,Confidence,Lift
0,{P1} -> {04X},"(P1,)","(04X,)",0.040396,0.108090,2.653256
1,{04X} -> {P1},"(04X,)","(P1,)",0.040396,0.991584,2.653256
2,{P1} -> {08X},"(P1,)","(08X,)",0.035027,0.093725,2.642423
3,{08X} -> {P1},"(08X,)","(P1,)",0.035027,0.987535,2.642423
4,{P2} -> {19X},"(P2,)","(19X,)",0.030696,0.173483,5.564910
5,{19X} -> {P2},"(19X,)","(P2,)",0.030696,0.984660,5.564910
6,{26Y} -> {P3},"(26Y,)","(P3,)",0.030516,0.963258,2.251049
7,{30X} -> {P3},"(30X,)","(P3,)",0.030047,0.974824,2.278078
8,{30Y} -> {P3},"(30Y,)","(P3,)",0.030615,0.976965,2.283081
9,{415} -> {P3},"(415,)","(P3,)",0.033205,0.466827,1.090934


## 4.2 Conviction指标

In [14]:
def conviction(self):
    eps = 10e-10  # Avoid zero division
    prob_not_rhs = 1 - self.count_rhs / self.num_transactions
    prob_not_rhs_given_lhs = 1 - self.confidence
    return prob_not_rhs / (prob_not_rhs_given_lhs + eps)

In [15]:
rules_df['Conviction'] = list(map(lambda x: conviction(x), rules))
rules_df

,Rules,Left,Right,Support,Confidence,Lift,Conviction
0,{P1} -> {04X},"(P1,)","(04X,)",0.040396,0.108090,2.653256,1.075514
1,{04X} -> {P1},"(04X,)","(P1,)",0.040396,0.991584,2.653256,74.411558
2,{P1} -> {08X},"(P1,)","(08X,)",0.035027,0.093725,2.642423,1.064280
3,{08X} -> {P1},"(08X,)","(P1,)",0.035027,0.987535,2.642423,50.242729
4,{P2} -> {19X},"(P2,)","(19X,)",0.030696,0.173483,5.564910,1.172179
5,{19X} -> {P2},"(19X,)","(P2,)",0.030696,0.984660,5.564910,53.654136
6,{26Y} -> {P3},"(26Y,)","(P3,)",0.030516,0.963258,2.251049,15.570459
7,{30X} -> {P3},"(30X,)","(P3,)",0.030047,0.974824,2.278078,22.723734
8,{30Y} -> {P3},"(30Y,)","(P3,)",0.030615,0.976965,2.283081,24.835625
9,{415} -> {P3},"(415,)","(P3,)",0.033205,0.466827,1.090934,1.072982
